# 定期券買ったほうがいいのかな？
在宅勤務が多かったうちは回数券で安く済ませてたけど、最近出勤する用事が増えてしまって交通費の出費が厳しくなってきた。<br>
ので、向こう半年のうち何日出勤するようならば定期券を買うべきなのか損益分岐点を求めてみる。<br>
わざわざNotebookにしたのはGitHubにプッシュするネタが欲しかったからです。

### 参考にした記事・サイト
- https://holidays-jp.github.io/
- https://qiita.com/dkugi/items/8c32cc481b365c277ec2
- https://qiita.com/calderarie/items/0ef921b476911a55148d
- https://www.techscore.com/blog/2019/12/16/better-way-handling-json-data-in-python/
- https://techacademy.jp/magazine/15826

In [1]:
import json, datetime, requests, pickle
from datetime import timedelta
from dateutil.relativedelta import relativedelta

from_ = datetime.date.today()
until_ = from_ + relativedelta(months=6)

# 消化予定の有給
paid_holiday = 8

# 回数券 vs 定期券
one_way_ticket = 280
one_way_suica = 272
coupon_ticket = 2800 / 11
six_month_commuter_pass = 53950

In [2]:
def dateRange(from_: datetime, until_:datetime):
    diff = (until_ - from_).days + 1
    return (from_ + timedelta(i) for i in range(diff))

In [3]:
def isHoliday(date: datetime):
    date = str(date)
    
    try:
        with open('holidays.pkl', 'rb') as f:
            holidays = pickle.load(f)       
    except EOFError:
        url = 'https://holidays-jp.github.io/api/v1/date.json'
        response = requests.get(url)
        holidays = response.json()
        with open('holidays.pkl', 'wb') as f:
            pickle.dump(holidays, f)
   
    return True if holidays.get(date) != None else False

In [4]:
def getWorkdays(from_: datetime, until_:datetime):
    return sum([not isHoliday(date) for date in dateRange(from_, until_) if date.weekday() < 5])

In [5]:
def getCost(workdays, cost):
    return workdays * cost

In [6]:
def main():
    workdays = getWorkdays(from_, until_) - paid_holiday
    n_weeks = ((until_ - from_) / 7).days
    
    for wfh_freq in [0, 0.5, 1, 2, 3]: # per week
        wfo_days = workdays - int(n_weeks * wfh_freq)
        print('◆週{:}日在宅勤務を行う場合（出勤日数：{:}日）'.format(wfh_freq, workdays))
        print('\t切符\t：{:,.1f}円 ({:,.1f}円)'.format(wfo_days*one_way_ticket*2, six_month_commuter_pass-wfo_days*one_way_ticket*2))
        print('\tSuica\t：{:,.1f}円 ({:,.1f}円)'.format(wfo_days*one_way_suica*2, six_month_commuter_pass-wfo_days*one_way_suica*2))
        print('\t回数券\t：{:,.1f}円 ({:,.1f}円)'.format(wfo_days*coupon_ticket*2, six_month_commuter_pass-wfo_days*coupon_ticket*2))
        print('\t定期券\t：{:,}円'.format(six_month_commuter_pass))
        print('\t')

In [7]:
main()

◆週0日在宅勤務を行う場合（出勤日数：117日）
	切符	：65,520.0円 (-11,570.0円)
	Suica	：63,648.0円 (-9,698.0円)
	回数券	：59,563.6円 (-5,613.6円)
	定期券	：53,950円
	
◆週0.5日在宅勤務を行う場合（出勤日数：117日）
	切符	：58,240.0円 (-4,290.0円)
	Suica	：56,576.0円 (-2,626.0円)
	回数券	：52,945.5円 (1,004.5円)
	定期券	：53,950円
	
◆週1日在宅勤務を行う場合（出勤日数：117日）
	切符	：50,960.0円 (2,990.0円)
	Suica	：49,504.0円 (4,446.0円)
	回数券	：46,327.3円 (7,622.7円)
	定期券	：53,950円
	
◆週2日在宅勤務を行う場合（出勤日数：117日）
	切符	：36,400.0円 (17,550.0円)
	Suica	：35,360.0円 (18,590.0円)
	回数券	：33,090.9円 (20,859.1円)
	定期券	：53,950円
	
◆週3日在宅勤務を行う場合（出勤日数：117日）
	切符	：21,840.0円 (32,110.0円)
	Suica	：21,216.0円 (32,734.0円)
	回数券	：19,854.5円 (34,095.5円)
	定期券	：53,950円
	


# 結論
<s>私の場合、だいたい週2.5日が分岐点で、月の半分以上在宅勤務すれば定期券を買わなくとも元が取れそうでした。<br>
そして今のペースだと赤字になることが判明したので今度定期買うことにしました。</s>

週末を考慮したら途端に定期券の優位性が消失しました。週1日ペースで在宅勤務をすれば定額切符ですら元が取れそうです。<br>
ここにさらに大晦日の会社指定休日も加味すると出勤日数は減るはずなので、どうやら定期券はもはや必要なさそうです。回数券 is all you need.<br>
一つ要望を出すとすれば、ぜひ都営交通並びにPasmoにはモバイル回数券を導入していただきたいです。